In [1]:

#Description: This program predicts the price of Bitcoin for the next 30 days

#Data Source: https://www.blockchain.com/charts/market-price?
# https://towardsdatascience.com/bitcoin-price-prediction-using-lstm-9eb0938c22bd

In [81]:
import numpy as np 
import pandas as pd

In [63]:
# Load the data set from the csv file using pandas
data = pd.read_csv('BitcoinPrice.csv')
#Store the data into the variable df
df = pd.read_csv('BitcoinPrice.csv')
df.head(7)

,Date,Price
0,"Jul 31, 2017",2763.24
1,"Jul 30, 2017",2724.39
2,"Jul 29, 2017",2807.02
3,"Jul 28, 2017",2679.73
4,"Jul 27, 2017",2538.71
5,"Jul 26, 2017",2577.77
6,"Jul 25, 2017",2757.50


In [64]:
#Remove the Date column
df.drop(['Date'], 1, inplace=True)

In [65]:
#Show the first 7 rows of the new data set
df.head(7)


,Price
0,2763.24
1,2724.39
2,2807.02
3,2679.73
4,2538.71
5,2577.77
6,2757.50


In [66]:

#A variable for predicting 'n' days out into the future
prediction_days = 30 #n = 30 days

#Create another column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Price']].shift(-prediction_days)

In [67]:

#Show the first 7 rows of the new data set
df.head(7)

,Price,Prediction
0,2763.24,2492.60
1,2724.39,2539.24
2,2807.02,2567.56
3,2679.73,2553.03
4,2538.71,2478.45
5,2577.77,2590.57
6,2757.50,2607.25


In [68]:
#Show the last 7 rows of the new data set
df.tail(7)


,Price,Prediction
1549,98.10,NaN
1550,106.25,NaN
1551,116.38,NaN
1552,139.00,NaN
1553,144.00,NaN
1554,134.44,NaN
1555,135.30,NaN


In [69]:
#CREATE THE INDEPENDENT DATA SET (X)

# Convert the dataframe to a numpy array and drop the prediction column
X = np.array(df.drop(['Prediction'],1))

#Remove the last 'n' rows where 'n' is the prediction_days
X= X[:len(df)-prediction_days]
print(X)

[[2763.24]
 [2724.39]
 [2807.02]
 ...
 [ 132.25]
 [ 129.  ]
 [ 129.77]]


In [70]:
#CREATE THE DEPENDENT DATA SET (y)

# Convert the dataframe to a numpy array (All of the values including the NaN's)
y = np.array(df['Prediction'])

# Get all of the y values except the last 'n' rows
y = y[:-prediction_days]
print(y)

[2492.6  2539.24 2567.56 ...  144.    134.44  135.3 ]


In [71]:
# Split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [72]:

# Set prediction_days_array equal to the last 30 rows of the original data set from the price column
prediction_days_array = np.array(df.drop(['Prediction'],1))[-prediction_days:]
print(prediction_days_array)

[[133.5 ]
 [131.99]
 [133.1 ]
 [126.3 ]
 [123.8 ]
 [122.89]
 [122.02]
 [122.5 ]
 [123.21]
 [123.5 ]
 [118.21]
 [114.22]
 [111.4 ]
 [117.98]
 [114.82]
 [115.64]
 [117.7 ]
 [112.8 ]
 [113.2 ]
 [109.6 ]
 [112.25]
 [115.98]
 [112.9 ]
 [ 98.1 ]
 [106.25]
 [116.38]
 [139.  ]
 [144.  ]
 [134.44]
 [135.3 ]]


In [78]:
from sklearn.svm import SVR
# Create and train the Support Vector Machine (Regression) using the radial basis function
svr_rbf = SVR(kernel='rbf',C=1e3, gamma=0.00001)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [74]:
# Testing Model: Score returns the accuracy of the prediction. 
# The best possible score is 1.0
svr_rbf_confidence = svr_rbf.score(x_test, y_test)
print("svr_rbf accuracy: ", svr_rbf_confidence)

svr_rbf accuracy:  -0.06781569111191743


In [75]:
# Print the predicted value
svm_prediction = svr_rbf.predict(x_test)
print(svm_prediction)

print()

#Print the actual values
print(y_test)

[431.42786135 434.62894653 432.37705204 431.56042857 429.62327411
 432.91668561 432.76147933 433.16360124 428.0382829  435.12128613
 436.06208128 432.66229898 433.00682102 435.57504231 435.45616605
 432.07713074 435.56343466 430.61228309 424.84167969 427.42672286
 432.41483178 431.24756962 434.62967623 432.41825107 434.46952482
 433.43492997 434.42337177 432.44095236 438.19608453 425.016571
 424.59862073 432.55695484 429.99362663 425.63024093 434.99085344
 436.05949121 432.36352183 432.39398611 436.32149048 415.04346327
 438.14707662 424.74649938 415.59786659 431.34839681 424.86623877
 427.5581882  436.81666022 422.1586313  432.62313835 430.55386023
 436.94735769 425.70247464 426.63734365 428.10875402 429.49731553
 431.50955898 432.37705204 432.70787101 431.52865461 435.12502272
 432.37705204 432.37705204 432.37705204 435.27684338 433.69951536
 431.71897111 432.78082629 433.75895919 436.52374078 435.98586365
 433.26356791 434.39599693 435.29883413 430.59716151 433.16360124
 425.2264943

In [76]:
# Print the model predictions for the next 'n=30' days
svm_prediction = svr_rbf.predict(prediction_days_array)
print(svm_prediction)

[428.95837611 426.70571766 428.25255177 427.03502031 431.03842537
 429.11209767 426.30540442 427.57317116 430.16747183 430.75652239
 429.70309632 432.19804145 429.90586856 430.44327739 432.35722077
 432.37642189 431.22795573 430.29364251 430.86246352 427.01510023
 430.07475916 432.37490365 430.40630782 428.3100709  426.87045434
 432.3622405  430.49681123 432.2278659  428.90172552 427.35438603]


In [77]:

#Print the actual price for the next 'n' days, n=prediction_days=30 
df.tail(prediction_days)

,Price,Prediction
1526,133.50,NaN
1527,131.99,NaN
1528,133.10,NaN
1529,126.30,NaN
1530,123.80,NaN
1531,122.89,NaN
1532,122.02,NaN
1533,122.50,NaN
1534,123.21,NaN
1535,123.50,NaN
